In [9]:
%matplotlib inline

import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [10]:
# Download mnist data

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
# Create model inputs

def model_inputs(discriminator_dim, generator_dim):
    discriminator_inputs = tf.placeholder(
        tf.float32,
        shape=[None, discriminator_dim],
        name='discriminator_input'
    )
    generator_inputs = tf.placeholder(
        tf.float32,
        shape=[None, generator_dim],
        name='generator_input'
    )
    return discriminator_inputs, generator_inputs

In [12]:
# Create generator

def generator(_input, out_dim, num_units=128, alpha=0.01):
    with tf.variable_scope('generator', reuse=True):
        hidden_layer = tf.layers.dense(_input, num_units)
        leaky_relu = tf.maximum(hidden_layer * alpha, hidden_layer)
        
        logits = tf.layers.dense(leaky_relu, out_dim)
        return tf.tanh(logits)

In [13]:
# Create discriminator

def discriminator(_input, num_units=128, alpha=0.01):
    with tf.variable_scope('discriminator', reuse=True):
        hidden_layer = tf.layers.dense(_input, num_units)
        leaky_relu = tf.maximum(hidden_layer * alpha, hidden_layer)
        
        logits = tf.layers.dense(leaky_relu, 1)
        return tf.sigmoid(logits), logits

In [14]:
# Set hyperparameters

discriminator_input_size = 728
generator_input_size = 100

generator_hidden_size = discriminator_hidden_size = 128

alpha = 0.01


smooth = 0.1